In [53]:
import os
import urllib.request
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

In [33]:
client = Client("tcp://128.110.217.26:8786")
client

<Client: 'tcp://128.110.217.26:8786' processes=2 threads=32, memory=125.38 GiB>

In [34]:
data_url = "/users/prsridha/witdata/1percent.tsv"
feature_names = ["language", "page_url", "image_url", "page_title", "section_title", "hierarchical_section_title", "caption_reference_description", "caption_attribution_description", "caption_alt_text_description", "mime_type", "original_height", "original_width", "is_main_image", "attribution_passes_lang_id", "page_changed_recently", "context_page_description", "context_section_description"]
dtypes = {"language" : str, "page_url" : str, "image_url" : str, "page_title" : str, "section_title" : str, "hierarchical_section_title" : str, "caption_reference_description" : str, "caption_attribution_description" : str, "caption_alt_text_description" : str, "mime_type" : str, "original_height" : np.int16, "original_width" : np.int16, "is_main_image" : bool, "attribution_passes_lang_id" : bool, "page_changed_recently" : bool, "context_page_description" : str, "context_section_description" : str }
dataset = pd.read_csv(data_url, names=feature_names, dtype=dtypes, sep='\t', header=0)
en = dataset[dataset.language == 'en']

In [35]:
df0 = dd.from_pandas(en, npartitions=2)
df = df0.sample(frac=0.01)

In [36]:
shuffled_df = df.sample(frac=1)
new_df = shuffled_df.repartition(npartitions=2)
new_df.head()

,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description
10427,en,https://en.wikipedia.org/wiki/Floriade_2012,http://upload.wikimedia.org/wikipedia/commons/...,Floriade 2012,NaN,Floriade 2012,Floriade theater,English: Floriade Theatre at the Floriade 2012...,NaN,image/jpeg,1650,2200,True,False,True,The Floriade 2012 was a Dutch horticultural ex...,The Floriade 2012 was a Dutch horticultural ex...
98019,en,https://en.wikipedia.org/wiki/Lichterfelde_(Be...,https://upload.wikimedia.org/wikipedia/commons...,Lichterfelde (Berlin),NaN,Lichterfelde (Berlin),"""Emisch Haus"" in Lichterfelde West",Deutsch: Villenkolonie Berlin Lichterfelde Wes...,"""Emisch Haus"" in Lichterfelde West",image/jpeg,457,640,True,False,False,Lichterfelde is a locality in the borough of S...,Lichterfelde is a locality in the borough of ...
124115,en,https://en.wikipedia.org/wiki/Charles_Reynolds...,https://upload.wikimedia.org/wikipedia/commons...,Charles Reynolds (cleric),Grave slab in Rome,Charles Reynolds (cleric) / Death / Grave slab...,NaN,La façade de Saint-Jean-de-Latran La basilique...,NaN,image/jpeg,1200,1515,False,False,True,"Charles Reynolds was an Irish Catholic cleric,...",Reynolds' grave-slab is in the cloister of the...
109773,en,https://en.wikipedia.org/wiki/Blue_runner,https://upload.wikimedia.org/wikipedia/commons...,Blue runner,NaN,Blue runner,NaN,"Caranx crysos (Mitchill). Runner. In: ""The Fis...",NaN,image/jpeg,335,785,True,True,False,"The blue runner, also known as the bluestripe ...","The blue runner (Caranx crysos), also known as..."
98882,en,https://en.wikipedia.org/wiki/Dreambox,https://upload.wikimedia.org/wikipedia/commons...,Dreambox,DM 7000,Dreambox / Dreambox models / DM 7000,DM 7000-S,A w:Dreambox 7000-S,NaN,image/jpeg,2513,3833,False,True,False,Dreambox is a series of Linux-powered DVB sate...,The DM 7000 is based around the IBM STB04500 c...


In [44]:
def pull(from_url, to_url):
    Path(to_url).mkdir(parents=True, exist_ok=True)
    
    urllib.request.urlretrieve(from_url, to_url)
    print("Pulled from {} to {}".format(from_url, to_url))

In [45]:
def resize_image(from_url):
    filename = from_url.split("/")[-1]
    to_path = "/users/prsridha/data/" + filename
    if not os.path.isfile(to_path):
        pull(from_url, to_path)
    
    im = Image.open(to_path)
    resized = im.resize((56,56))
    resized_np = np.array(resized)
    return resized_np

In [46]:
def identity(x):
    return x

In [51]:
new_col = new_df.image_url.map_partitions(lambda x: x.apply(lambda y: resize_image(str(x))), meta=('new_col', object))

In [52]:
new_col.compute()

NameError: name 'os' is not defined